# Original Objective function
$$
E(A_{i}) = \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} k_{ij} q_j \prod_{i' \in \{1, \ldots, m\}}^{\text{msg}} \left( k_{i'j} p_{i'} + 0^{k_{i'j}} \right)
$$

### Simplification assumption
We assume probobility of recieving a tag $q_{j}$ is equal to probobility of recieving message $p_{i}$  
$$q_{j} = p_{i} = p$$

### C1 : Using all tags
Each tag must verify at least one message
$$
\sum_{i=1}^{m} k_{ij} \geq 1 \quad \forall j \in \{1, 2, \ldots, \text{|T|}\}
$$
### C2: Using all messages
Each message must be signed with at least one tag
$$
\sum_{j=1}^{n} k_{ij} \geq 1 \quad \forall i \in \{1, 2, \ldots, \text{|M|}\}
$$

### C3: Define auxiliary varible for number of messages each tag signs
we define number of messages that each tag signs as an auxiliray variable to use later in transforming objective function to a linear model
$$
z_j = \sum_{i=1}^{n} k_{ij} \quad \forall j \in \{1, 2, \ldots, \text{tags}\}
$$
$$
z_j \geq 1 \quad \forall j \in \{1, 2, \ldots, \text{tags}\}
$$


### C4: Re-writing the product part in linear
After simplifying all probobilities to be equal the product part could be linearized using some auxiliary variable and constraints as follow:

$$
\prod_{i' \in \{1, \ldots, m\}}^{\text{msg}} \left( k_{i'j} p_{i'} + 0^{k_{i'j}} \right) =  \sum_{i' \in \{1, \ldots, m\}}^{\text{msg}} w_{ji'} p^{i'}
$$


Now the next constraint should be define to force the correcponding $w_{ji'} = 1$ which can be done as follow





1. **Link $ z[j] $ and $ y[j,i] $ using big-M constraints:**

$$
z[j] - i \leq M(1 - y[j,i])
$$

$$
i - z[j] \leq M(1 - y[j,i])
$$

2. **Ensure $ y[j,i] $ selects only one $ i $:**

$$
\sum_{i \in \{1, \ldots, m\}}^{\text{msg}} y[j,i] = 1
$$

3. **Link $ w[j,i] $ to $ y[j,i] $:**

$$
w[j,i] \leq y[j,i]
$$

$$
w[j,i] \geq y[j,i] - (1 - y[j,i])
$$

$$
\sum_{i \in \{1, \ldots, m\}}^{\text{msg}} w[j,i] = 1
$$


The final part is linearizing the product of the two binary variables $k_{ij}w_{j}$ which could be done this way:

$$x_{mij} \leq k_{ij} $$
$$x_{mij} \leq w_{ji'} $$
$$x_{mij} \geq k_{ij} + w_{ji'} -1 $$
$$x_{mij}\in \{0,1\}$$

## Simplified Objective function
With linearizing product part, the objective function could be rewritten as: 

$$
E(A_{i}) =\sum_{i \in \{1, \ldots, m\}}^{\text{msg}} \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} \sum_{i' \in \{1, \ldots, m\}}^{\text{msg}} x_{iji'} p^{i'+1}

$$


In [1]:
import numpy as np
import pandas as pd
from pulp import *

In [2]:

def max_expected_A(n_tags, m_msgs, p):
   
    # Create a PuLP problem with maximization
    prob = LpProblem("Maximize_Expected_A", LpMaximize)

    # Define the binary variables k indexed by (i, j)
    k = {(i, j): LpVariable(f'k_{i}_{j}', cat='Binary') for i in range(1, m_msgs+1) for j in range(1, n_tags+1)}

    # Define the binary variables w indexed by (j,i)
    w = {(j, i): LpVariable(f'w_{j}_{i}', cat='Binary') for j in range(1, n_tags+1) for i in range(1, m_msgs+1)}

    # Define the binary variables x indexed by (i, j)
    x = {(i, j, m): LpVariable(f'x_{i}_{j}_{m}', cat='Binary') for i in range(1, m_msgs+1) for j in range(1, n_tags+1) for m in range(1, m_msgs+1)}

    # Define the binary variables y indexed by (j, i)
    y = {(j, i): LpVariable(f'y_{j}_{i}', cat='Binary') for j in range(1, n_tags+1) for i in range(1, m_msgs+1)}

    # Define the continuous variables z indexed by j
    z = {j: LpVariable(f'z_{j}', cat='Integer', lowBound=0) for j in range(1, n_tags+1)}

    # Constraints
    # for j in range(1, n_tags+1):
    #     prob += lpSum(k[i, j] for i in range(1, m_msgs+1)) >= 1, f"Tag_{j}_constraint"

    # for i in range(1, m_msgs+1):
    #     prob += lpSum(k[i, j] for j in range(1, n_tags+1)) >= 1, f"Message_{i}_constraint"

    for j in range(1, n_tags+1):
        prob += z[j] == lpSum(k[i, j] for i in range(1, m_msgs+1)), f"Z_{j}_definition"
        #prob += z[j] >= 1, f"Z_{j}_non_zero"
        
    for j in range(1, n_tags+1):
        prob += lpSum(w[j, i] for i in range(1, m_msgs+1)) == 1, f"Sum_w_{j}_equals_1"

    for j in range(1, n_tags+1):
        for i in range(1, m_msgs+1):
            prob += z[j] - i <= M * (1 - y[j,i]), f"BigM_1_{j}_{i}"
            prob += i - z[j] <= M * (1 - y[j,i]), f"BigM_2_{j}_{i}"

    for j in range(1, n_tags+1):
        prob += lpSum(y[j, i] for i in range(1, m_msgs+1)) == 1, f"y_{j}_{i}_equals_one"
    
    for j in range(1, n_tags+1):
        prob += lpSum(w[j, i] for i in range(1, m_msgs+1)) == 1, f"w_{j}_{i}_equals_one"

    for j in range(1, n_tags+1):
        for i in range(1, m_msgs+1):
            prob += w[j, i]  <= y[j, i] , f"w_{j}_{i}_less_y_{j}_{i}"

    for j in range(1, n_tags+1):
        for i in range(1, m_msgs+1):
            prob += w[j,i] >= y[j, i] - (1 - y[j,i])  , f"y_{j}_{i}_w_{j}_{i}"


    for i in range(1, m_msgs+1):
        for j in range(1, n_tags+1):
            for m in range(1, m_msgs+1):
                prob += x[i,j, m] <= k[i, j], f"x_{i}_{j}_{m}_leq_k_{i}_{j}"
                prob += x[i,j, m] <= w[j,m], f"x_{i}_{j}_{m}_leq_w_{j}_{m}"
                prob += x[i,j, m] >= k[i, j] + w[j,m] - 1, f"x_{i}_{j}_{m}_geq_k_{i}_{j}_plus_w_{j}_{m}_minus_1"


    # Objective Function
    prob += lpSum(x[i,j,m]*P[m] for i in range(1, m_msgs+1) for j in range(1, n_tags+1)for m in range(1, m_msgs+1)), "Maximize_E_A"

    # Solve the problem
    #solver = pulp.PULP_CBC_CMD(timeLimit=120)
    #prob.solve(solver)
    prob.solve()

    # Extract the optimal values of k_{ij}
    k_values = np.zeros((m_msgs, n_tags), dtype=int)
    for i in range(1, m_msgs+1):
        for j in range(1, n_tags+1):
            k_values[i-1, j-1] = value(k[i, j])


    k_df = pd.DataFrame(k_values, columns=[f'k_{j}' for j in range(1, n_tags+1)], index=[f'Msg_{i}' for i in range(1, m_msgs+1)])
    k_df.columns = [f't{i}' for i in range(1,n_tags+1)]
    k_df.index = [f'm{i}' for i in range(1,m_msgs+1)]


    sum_row = k_df.sum()
    k_df.loc['Sum'] = sum_row


    # Print the variable values
    for v in prob.variables():
        print(f"{v.name} = {v.varValue}")

    # Print the objective value
    print(f"Objective value: {value(prob.objective)}")


    return k_df


In [3]:
n_tags = 4
n_msgs = 8
p = 0.7
P = {i: p**(i) for i in range(1, n_msgs+2)}  # p^i coefficients
M = 1000     # A sufficiently large number for the constraints

In [4]:
K = max_expected_A(n_tags, n_msgs, p)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/jv/12n64fl91y1d9jr6561fpx2c0000gn/T/10f84086c9384c828ddcf24d0f48e377-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/jv/12n64fl91y1d9jr6561fpx2c0000gn/T/10f84086c9384c828ddcf24d0f48e377-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 917 COLUMNS
At line 4066 RHS
At line 4979 BOUNDS
At line 5336 ENDATA
Problem MODEL has 912 rows, 356 columns and 2180 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 22.3533 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 252 strengthened rows, 128 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 479 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 485 strengthened rows, 0 

,t1,t2,t3,t4
m1,0,1,1,1
m2,1,0,0,0
m3,0,0,0,1
m4,1,1,1,0
m5,0,1,0,1
m6,1,0,1,0
m7,0,0,0,0
m8,0,0,0,0
Sum,3,3,3,3
